## 3장-4절. Neural Network
- 뇌의 신경망을 본 떠 만든 알고리즘
- 딥러닝의 기초가 되는 알고리즘이다.
- 중요 하이퍼파라미터
    - activation(활성함수)
        - ['identity’, ‘logistic’, ‘tanh’, ‘relu’], default=’relu’
    - solver
        - ['lbfgs', 'sgd', 'adam'],default=’adam’
        - 큰 세트에서는 adam, 작은 세트에서는 lbfgs 더 나은 성능
    - alpha
        - L2 패널티 매개변수
        - 작을수록 제약을 안두고 클수록 최대한 간단한 모델로 피팅한다.
    - hidden_layer_sizes
        - 튜플로 여러층 나타낼수 있고, 노드수를 지정해줄 수 있음
        - 예) hideen_layer_sizes=(5,2) 는 layer2개이고, 첫번째레이어의 노드 5, 두번째 레이어의 노드는 2인 것이다.
    - learning_rate
        - 가중치 업데이트에 대한 학습률 일정
        - [‘constant’, ‘invscaling’, ‘adaptive’], default=’constant’
    - max_iter
        - 최대 반복 횟수
     

In [27]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 데이터 불러오기

In [28]:
wine_red = pd.read_csv('./data/winequality-red.csv',sep=';')
wine_white = pd.read_csv('./data/winequality-white.csv',sep=';')

wine_red['color'] = 1
wine_white['color'] = 0

wine = pd.concat([wine_red,wine_white])
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [29]:
from sklearn.preprocessing import LabelEncoder

wine[['quality']] = wine[['quality']].apply(LabelEncoder().fit_transform)
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2,1


In [30]:
y = wine['color']
X = wine.drop(columns=['color'])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,stratify=y)

In [31]:
from sklearn.preprocessing import StandardScaler
ss_scaler = StandardScaler()

X_train_scaled = ss_scaler.fit_transform(X_train)
X_test_scaled = ss_scaler.transform(X_test)

## Neural Network 분석

### 01. 분류분석

In [32]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(5,2))
clf.fit(X_train_scaled, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), solver='lbfgs')

In [33]:
pred_train = clf.predict(X_train_scaled)
pred_test = clf.predict(X_test_scaled)

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(y_train,pred_train)

array([[3428,    0],
       [   9, 1110]], dtype=int64)

In [35]:
confusion_matrix(y_test,pred_test)

array([[1469,    1],
       [   6,  474]], dtype=int64)

In [37]:
print(accuracy_score(y_test,pred_test))

0.9964102564102564


### 01-2. gridsearch

In [40]:
from sklearn.model_selection import GridSearchCV

hyperparams = {
    'hidden_layer_sizes': np.arange(10,15),
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs','adam'],
    'alpha': 10.0 ** -np.arange(8, 10),
    'learning_rate': ['constant','adaptive']
}

gs = GridSearchCV(MLPClassifier(),
                 param_grid = hyperparams,
                 cv=3,
                 n_jobs=-1,
                 scoring='accuracy')

In [41]:
gs.fit(X_train_scaled,y_train)

GridSearchCV(cv=3, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': array([1.e-08, 1.e-09]),
                         'hidden_layer_sizes': array([10, 11, 12, 13, 14]),
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['lbfgs', 'adam']},
             scoring='accuracy')

In [44]:
print(gs.best_estimator_)
print(gs.best_params_)

MLPClassifier(alpha=1e-08, hidden_layer_sizes=11, learning_rate='adaptive')
{'activation': 'relu', 'alpha': 1e-08, 'hidden_layer_sizes': 11, 'learning_rate': 'adaptive', 'solver': 'adam'}


In [45]:
pred_test_gs = gs.predict(X_test_scaled)

accuracy_score(y_test,pred_test_gs)

0.9974358974358974

---
---

### 02. 회귀분석 

In [46]:
# 데이터 만들기
rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16))

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [48]:
from sklearn.neural_network import MLPRegressor

mlpr = MLPRegressor()
mlpr.fit(X_train,y_train)

MLPRegressor()

In [49]:
pred_test_mlpr = mlpr.predict(X_test)

In [50]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,pred_test_mlpr))

0.4067479357735299